<a href="https://colab.research.google.com/github/austiezr/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [x] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [x] Engineer at least two new features. (See below for explanation & ideas.)
- [x] Fit a linear regression model with at least two features.
- [x] Get the model's coefficients and intercept.
- [x] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [x] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [x] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [x] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

### Loading

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import plotly.graph_objects as go
import itertools

In [0]:
# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [604]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Cleaning and Feature Engineering

In [0]:
# reducing date to month/year

df['created'] = [x[:-12] for x in df['created']]

In [0]:
# setting numeric values to interest level

df['interest_level'] = df['interest_level'].map({'low' : 1, 'medium' : 2, 'high' : 3})

In [0]:
# total number of bedrooms/bathrooms

df['bed_bath'] = (lambda x, y: x + y)(df['bedrooms'], df['bathrooms'])

### Finding only features that return an $R^2$ value of at least .05 to create 'arbitrary_luxury_value' feature

In [608]:
reg = LinearRegression()
  
test = df[df['created'] == '2016-06']
train = df[((df['created'] == '2016-05') | (df['created'] == '2016-04'))]

notUsed = ['created', 'description', 'display_address', 'price', 
           'street_address', 'interest_level', 'arbitrary_luxury_value']

features = []

for column in df:
  if column not in notUsed:
    testX = test[[column]]
    trainX = train[[column]]
    testY = test['price']
    trainY = train['price']

    reg.fit(trainX, trainY)

    predY = reg.predict(testX)
    
    if r2_score(testY, predY) >= .05:
      features.append(column)

features

['bathrooms',
 'bedrooms',
 'longitude',
 'doorman',
 'fitness_center',
 'laundry_in_unit',
 'dining_room',
 'bed_bath']

In [0]:
'''
new feature that is a sum of all features returning at least .05 r2, divided by interest level, 
not including bedrooms/bathrooms/longitude
'''

df['arbitrary_luxury_value'] = sum((df['doorman'], df['fitness_center'], 
                                    df['laundry_in_unit'], df['dining_room'], 
                                    df['bed_bath']))/df['interest_level']

## Multi-Regression -- Best MAE = 676.92

In [0]:
def multiRegression(features, df=df, target='price', num=100):
  reg = LinearRegression()

  test = df[df['created'] == '2016-06']
  train = df[((df['created'] == '2016-05') | (df['created'] == '2016-04'))]
  testX = test[features]
  trainX = train[features]
  testY = test[target]
  trainY = train[target]
  
  reg.fit(trainX, trainY)

  predY = reg.predict(testX)

  estimate = predY
  coefficient = reg.coef_[0]

  print(f'Coefficients: {reg.coef_}')
  print(f'Mean Absolute Error: ${mean_absolute_error(testY, predY):,.2f}')
  print(f'Root Mean Squared Error: ${mean_squared_error(testY, predY, squared=False):,.2f}')
  print(f'Intercept: {reg.intercept_}')
  print(f'R^2 Coefficient of Determination: {r2_score(testY, predY)}')
  
  if len(features)<3:
    fig = go.Figure(data=[go.Scatter3d(x=testX.iloc[:, 0],
                                      y=testX.iloc[:, 1],
                                      z=testY,
                                      mode='markers')])
  
    '''
    Plots a pseudo-plane of best fit - faster and easier but isn't as readable
    as Ryan's solution below.

    fig.add_trace(go.Scatter3d(x=testX.iloc[:, 0],
                              y=testX.iloc[:, 1],
                              z=predY))
    '''
  
    xmin, xmax = testX.iloc[:, 0].min(), testX.iloc[:, 0].max()
    ymin, ymax = testX.iloc[:, 1].min(), testX.iloc[:, 1].max()
    xcoords = np.linspace(xmin, xmax, num)
    ycoords = np.linspace(ymin, ymax, num)
    coords = list(itertools.product(xcoords, ycoords))

    predictions = reg.predict(coords)
    Z = predictions.reshape(num, num).T
  
    xmin, xmax = testX.iloc[:, 0].min(), testX.iloc[:, 0].max()
    ymin, ymax = testX.iloc[:, 1].min(), testX.iloc[:, 1].max()
    xcoords = np.linspace(xmin, xmax, num)
    ycoords = np.linspace(ymin, ymax, num)
    coords = list(itertools.product(xcoords, ycoords))

    predictions = reg.predict(coords)
    Z = predictions.reshape(num, num).T

    fig.add_trace(go.Surface(x=xcoords, 
                           y=ycoords, 
                           z=Z))

    fig.show()

### Regression plotted with arbitrary_luxury_value, bathrooms

In [611]:
multiRegression(['arbitrary_luxury_value', 'bathrooms'])

Coefficients: [ 406.00904664 1472.90070507]
Mean Absolute Error: $751.83
Root Mean Squared Error: $1,121.15
Intercept: 545.0851548590958
R^2 Coefficient of Determination: 0.5955664663197989


### Regression using arbitrary_luxury_value, number of bathrooms, bedrooms, longitude -- MAE 676.92


In [612]:
multiRegression(['arbitrary_luxury_value', 'longitude', 'bathrooms', 'bedrooms'])

Coefficients: [   314.86094385 -12857.74210743   1428.65020638    220.16261042]
Mean Absolute Error: $676.92
Root Mean Squared Error: $1,055.41
Intercept: -950579.8639712036
R^2 Coefficient of Determination: 0.6416083199385816


## Prediction using multiple features

In [0]:
def multiPredict(features, featureValues, df=df, target='price'):
  reg = LinearRegression()

  test = df[df['created'] == '2016-06']
  train = df[((df['created'] == '2016-05') | (df['created'] == '2016-04'))]
  testX = test[features]
  trainX = train[features]
  testY = test[target]
  trainY = train[target]
  
  reg.fit(trainX, trainY)

  predY = reg.predict([featureValues])

  estimate = predY[0]
  coefficient1 = reg.coef_[0]
  coefficient2 = reg.coef_[1]
  result = f'${estimate:,.2f} estimated price for {featureValues[0]} {features[0]}, {featureValues[1]} {features[1]}.'
  explanation = f'Each increase in {features[0]} adds ${coefficient1:,.2f}, and each increase in {features[1]} adds ${coefficient2:,.2f}.'
  print(f'{result} \n{explanation}')

In [614]:
multiPredict(['arbitrary_luxury_value', 'bathrooms'], [5, 2])

$5,520.93 estimated price for 5 arbitrary_luxury_value, 2 bathrooms. 
Each increase in arbitrary_luxury_value adds $406.01, and each increase in bathrooms adds $1,472.90.
